In [4]:
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
import time

In [5]:
import scipy.optimize as op
import george as gg
import george.kernels as kr

In [6]:
def gen_images_with_GP(file_name, path_to_read, path_to_export,error_margin, points, show_fig, width, resolution):
    #file_name must be like: DES_SN123456.DAT
    file_id = file_name[4:12]
    file_read = read_sn(path_to_read + file_name)
    sn_type = file_read['SIM_COMMENT'][3]
    #TODO, add verification here, add type verification and also if it has 4 bands, 
#if it does not, return 0
    GP_Function(file_read['df'], file_id, sn_type, path_to_export,error_margin, points, show_fig, width , resolution )

def GP_Function(df_all, file_id, sn_type, path_to_export,error_margin, points, show_fig, width, resolution):
    
    x = np.linspace(0, df_all.MJD.max() + (df_all.MJD.max())*0.1, 200)
    
    data = {band: df[['MJD', 'FLUXCAL', 'FLUXCALERR']].values for band, df in df_all.groupby('FLT')}
    colors = {'g': 'g', 'r': 'r', 'i': 'y', 'z': 'b'}

    for band, dat in data.items():
        gp = gg.GP((500**2)*kr.ExpSquaredKernel(metric=20**2), fit_mean=True)
        gp.compute(dat[:,0], dat[:,2])

        def nll(p):
            gp.set_parameter_vector(p)
            ll = gp.log_likelihood(dat[:,1], quiet=True)
            return -ll if np.isfinite(ll) else 1e25
        
        def grad_nll(p):
            gp.set_parameter_vector(p)
            return -gp.grad_log_likelihood(dat[:,1], quiet=True)
            
        p0 = gp.get_parameter_vector()
        results = op.minimize(nll, p0, jac=grad_nll, method="L-BFGS-B")
          
        mu, var = gp.predict(dat[:,1], x, return_var=True)
        std = np.sqrt(var)
        
        matplotlib.use('Agg')
        plt.ioff()
        fig = plt.figure(num=None, figsize=(8, 5), dpi = resolution, facecolor='w', edgecolor='w')
        #plt.ylim(bottom = 1)
        #plt.xlim(-0, 250)
        if error_margin == True:
            plt.fill_between(x, mu-std, mu+std, alpha=.1, color= 'k')
        plt.plot(x, mu, color= 'k', linewidth = width)
        if points == True:
            plt.errorbar(dat[:,0], dat[:,1], yerr=dat[:,2], fmt='o', color= 'k')
        plt.axis('off')
        plt.savefig(path_to_export + file_id + '_'+ sn_type +'_'+ band + '.png')
        if show_fig == False:
            plt.close(fig)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes, dim_x, dim_y,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)
    
    fig, ax = plt.subplots(figsize = (dim_x,dim_y))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
#DRAFT

#def gen_images_with_GP(file_name, path_to_read, path_to_export):
#    #file_name must be like: DES_SN123456.DAT
#    file_id = file_name[4:12]
#    file_read = read_sn(path_to_read + file_name)
#    sn_type = file_read['SIM_COMMENT'][3]
#    #TODO, add verification here, add type verification and also if it has 4 bands, 
##if it does not, return 0
#    GP_Function(file_read['df'], file_id, sn_type, path_to_export)
#
#def GP_Function(df_all, file_id, sn_type, path_to_export):
#    
#    x = np.linspace(0, df_all.MJD.max() + (df_all.MJD.max())*0.1, 200)
#    
#    data = {band: df[['MJD', 'FLUXCAL', 'FLUXCALERR']].values for band, df in df_all.groupby('FLT')}
#    colors = {'g': 'g', 'r': 'r', 'i': 'y', 'z': 'b'}
#
#    plt.figure(num=None, figsize=(16, 10), dpi=80, facecolor='w', edgecolor='k')
#    for band, dat in data.items():
#        #a diferença eh que no dat[:,0], dat[:,2] ele nao especifica qual eh a banda
#        #entao creio que faca pra todo mundo
#        gp = gg.GP((500**2)*kr.ExpSquaredKernel(metric=20**2), fit_mean=True)
#        gp.compute(dat[:,0], dat[:,2])
#      
#          # Define the objective function (negative log-likelihood in this case).
#        def nll(p):
#            gp.set_parameter_vector(p)
#            ll = gp.log_likelihood(dat[:,1], quiet=True)
#            return -ll if np.isfinite(ll) else 1e25
#        
#          # And the gradient of the objective function.
#        def grad_nll(p):
#            gp.set_parameter_vector(p)
#            return -gp.grad_log_likelihood(dat[:,1], quiet=True)
#            
#        p0 = gp.get_parameter_vector()
#        results = op.minimize(nll, p0, jac=grad_nll, method="L-BFGS-B")
#          
#        mu, var = gp.predict(dat[:,1], x, return_var=True)
#        std = np.sqrt(var)
#        plt.figure(num=None, figsize=(16, 10), dpi=80, facecolor='w', edgecolor='k')
#        
#        #plt.ylim(bottom = 1)
#        #plt.xlim(-0, 250)
#        #aparentemente os mu, std ficou tudo sendo mais de uma dimensao
#        #carregando as informacoes das 4 curvas a interpolar
#        
#        #daqui pra baixo so plot
#        plt.fill_between(x, mu-std, mu+std, alpha=.3, color=colors[band[-1]])
#        plt.plot(x, mu, color=colors[band[-1]])
#        plt.errorbar(dat[:,0], dat[:,1], yerr=dat[:,2], fmt='o', color=colors[band[-1]])
#        plt.axis('off')
#        plt.savefig(path_to_export + file_id + '_'+ sn_type +'_'+ band + '.png')
#          
#        #plt.xlabel('time (days)')
#        #plt.ylabel('flux')
#        #plt.plot(x, np.array([0 for i in range(len(x))]) , 'k-') 